# Analiza zmiany wartości mid price na podstawie Limit Order Book

Będziemy LOBy analizować 


In [1]:
from imbalance import prep_data
from parser import parse
from utils import confusion_matrix
from warnings import filterwarnings
filterwarnings('ignore')


file_locs = ['data/OrderBookSnapshots.9061.csv', 'data/OrderBookSnapshots.9062.csv', 'data/OrderBookSnapshots.9063.csv',
             'data/OrderBookSnapshots.9064.csv', 'data/OrderBookSnapshots.9065.csv']

## Imbalance

No to z czym sie będziemy porównywać to imbalance z pracy (link)

In [3]:
from sklearn import linear_model


for file_name in file_locs:
    X_train, Y_train, X_test, Y_test = prep_data(parse(file_name))
    classifier = linear_model.SGDClassifier(loss="log", alpha=0.1, max_iter=3000, tol=0, shuffle=False)
    classifier.fit(X_train, Y_train)
    Y_check = classifier.predict(X_test)
    print(classifier.score(X_test,Y_test))
    print(confusion_matrix(Y_check, Y_test))
    print()

0.5370569280343717
[[287 274]
 [157 213]]

0.5549626467449307
[[231 201]
 [216 289]]

0.582039911308204
[[237 149]
 [228 288]]

0.5345005149330587
[[151 114]
 [338 368]]

0.5603674540682415
[[167 108]
 [227 260]]



## Klasyfikator oparty na sieci neuronowej

No nie wiem czy coś tu pisać

### Preprocessing

No kubełki czy coś

### Model

Słaby jakiś

## Wyniki

no nie działa

Dziękujemy za uwagę